# Машинное обучение, ФКН ВШЭ

## Практическое задание 8. Метод опорных векторов и аппроксимация ядер

### Общая информация

Дата выдачи: 28.01.2022

Мягкий дедлайн: 23:59MSK 14.02.2022

Жесткий дедлайн: 23:59MSK 17.02.2022

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимальная оценка за работу (без учёта бонусов) — 10 баллов.

Сдавать задание после указанного жёсткого срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

### Формат сдачи
Задания сдаются через систему anytask. Посылка должна содержать:
* Ноутбук homework-practice-08-random-features-Username.ipynb

Username — ваша фамилия и имя на латинице именно в таком порядке

### О задании

На занятиях мы подробно обсуждали метод опорных векторов (SVM). В базовой версии в нём нет чего-то особенного — мы всего лишь используем специальную функцию потерь, которая не требует устремлять отступы к бесконечности; ей достаточно, чтобы отступы были не меньше +1. Затем мы узнали, что SVM можно переписать в двойственном виде, который, позволяет заменить скалярные произведения объектов на ядра. Это будет соответствовать построению модели в новом пространстве более высокой размерности, координаты которого представляют собой нелинейные модификации исходных признаков.

Ядровой SVM, к сожалению, довольно затратен по памяти (нужно хранить матрицу Грама размера $d \times d$) и по времени (нужно решать задачу условной оптимизации с квадратичной функцией, а это не очень быстро). Мы обсуждали, что есть способы посчитать новые признаки $\tilde \varphi(x)$ на основе исходных так, что скалярные произведения этих новых $\langle \tilde \varphi(x), \tilde \varphi(z) \rangle$ приближают ядро $K(x, z)$.

Мы будем исследовать аппроксимации методом Random Fourier Features (RFF, также в литературе встречается название Random Kitchen Sinks) для гауссовых ядер. Будем использовать формулы, которые немного отличаются от того, что было на лекциях (мы добавим сдвиги внутрь тригонометрических функций и будем использовать только косинусы, потому что с нужным сдвигом косинус превратится в синус):
$$\tilde \varphi(x) = (
\cos (w_1^T x + b_1),
\dots,
\cos (w_n^T x + b_n)
),$$
где $w_j \sim \mathcal{N}(0, 1/\sigma^2)$, $b_j \sim U[-\pi, \pi]$.

На новых признаках $\tilde \varphi(x)$ мы будем строить любую линейную модель.

Можно считать, что это некоторая новая парадигма построения сложных моделей. Можно направленно искать сложные нелинейные закономерности в данных с помощью градиентного бустинга или нейронных сетей, а можно просто нагенерировать большое количество случайных нелинейных признаков и надеяться, что быстрая и простая модель (то есть линейная) сможет показать на них хорошее качество. В этом задании мы изучим, насколько работоспособна такая идея.

### Алгоритм

Вам потребуется реализовать следующий алгоритм:
1. Понизить размерность выборки до new_dim с помощью метода главных компонент.
2. Для полученной выборки оценить гиперпараметр $\sigma^2$ с помощью эвристики (рекомендуем считать медиану не по всем парам объектов, а по случайному подмножеству из где-то миллиона пар объектов): $$\sigma^2 = \text{median}_{i, j = 1, \dots, \ell, i \neq j} \left\{\sum_{k = 1}^{d} (x_{ik} - x_{jk})^2 \right\}$$
3. Сгенерировать n_features наборов весов $w_j$ и сдвигов $b_j$.
4. Сформировать n_features новых признаков по формулам, приведённым выше.
5. Обучить линейную модель (логистическую регрессию или SVM) на новых признаках.
6. Повторить преобразования (PCA, формирование новых признаков) к тестовой выборке и применить модель.

Тестировать алгоритм мы будем на данных Fashion MNIST. Ниже код для их загрузки и подготовки.

In [1]:
from keras.datasets import fashion_mnist

from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier

import numpy as np
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
from optuna.samplers import TPESampler

from matplotlib import pyplot as plt

MAIN_SEED = 42

rnd = np.random.default_rng(MAIN_SEED)

In [2]:
(x_train_pics, y_train), (x_test_pics, y_test) = fashion_mnist.load_data()
x_train = x_train_pics.reshape(x_train_pics.shape[0], -1)
x_test = x_test_pics.reshape(x_test_pics.shape[0], -1)

In [3]:
X_train = x_train / x_train.max()
X_test = x_test / x_train.max()

__Задание 1. (5 баллов)__

Реализуйте алгоритм, описанный выше. Можете воспользоваться шаблоном класса ниже или написать свой интерфейс.

Ваша реализация должна поддерживать следующие опции:
1. Возможность задавать значения гиперпараметров new_dim (по умолчанию 50) и n_features (по умолчанию 1000).
2. Возможность включать или выключать предварительное понижение размерности с помощью метода главных компонент.
3. Возможность выбирать тип линейной модели (логистическая регрессия или SVM с линейным ядром).

Протестируйте на данных Fashion MNIST, сформированных кодом выше. Если на тесте у вас получилась доля верных ответов не ниже 0.84 с гиперпараметрами по умолчанию, то вы всё сделали правильно.

In [30]:
from sklearn.base import BaseEstimator, TransformerMixin


class RFFPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, n_features=1000, new_dim=50, use_PCA=True, classifier='logreg'):
        """        
        Implements pipeline, which consists of PCA decomposition,
        Random Fourier Features approximation and linear classification model.
        
        n_features, int: amount of synthetic random features generated with RFF approximation.

        new_dim, int: PCA output size.ы
        
        use_PCA, bool: whether to include PCA preprocessing.
        
        classifier, string: either 'svm' or 'logreg', a linear classification model to use on top of pipeline.
        
        Feel free to edit this template for your preferences.    
        """
        self.n_features = n_features
        self.use_PCA = use_PCA
        self.new_dim = new_dim
        self.classifier = classifier
        
    def fit(self, X, y):
        """
        Fit all parts of algorithm (PCA, RFF, Classification) to training set.
        """
        if self.use_PCA:
            self.pca= PCA(n_components=self.new_dim)
            X = self.pca.fit_transform(X)

        # Calculating sigma2

        sigma_values = np.array([])

        for _ in range(int(1e5)):
            i, j = rnd.integers(X.shape[0], size=2)
            if i == j:
                i -= 1
            sigma_values = np.append(sigma_values, np.sum((X[i] - X[j])) ** 2)
        
        sigma2 = sigma_values.mean()
        #for _ in range(int(1e5)):
        #    i, j = np.random.randint(0, len(X_train), size=2)
        #    if i == j:
        #        i = len(X_train) - i - 1
        #        sigma_values = np.append(sigma_values, sum((X_train[i] - X_train[j]) ** 2))
        #sigma2 = np.median(sigma_values)
        

        print(f'Sigma2 calculated {sigma2}')

        # Calculating weights

        self.w = rnd.normal(0, 1/np.sqrt(sigma2), (X.shape[1], self.n_features))
        self.b = rnd.uniform(-np.pi, np.pi, self.n_features)

        print('Weights calculated')

        # Calculating phi matrix

        phi_matrix = np.cos(X @ self.w + self.b)
        print(phi_matrix.shape)

        print('Phi matrix calculated')
        
        if self.classifier == 'svm':
            self.class_ = SVC(kernel='linear')
        elif self.classifier == 'logreg':
            self.class_ = LogisticRegression()
        
        self.class_.fit(phi_matrix, y)
        return self.w
        

    def predict_proba(self, X):
        """
        Apply pipeline to obtain scores for input data.
        """
        if self.use_PCA:
            X = self.pca.transform(X)

        phi_matrix = np.cos(X @ self.w + self.b)

        return self.class_.predict_proba(phi_matrix)
        
    def predict(self, X):
        """
        Apply pipeline to obtain discrete predictions for input data.
        """
        if self.use_PCA:
            X = self.pca.transform(X)

        phi_matrix = np.cos(X @ self.w + self.b)

        return self.class_.predict(phi_matrix)


Воспользуемся сплитом, чтобы уменьшить выборку для обучения и при этом не потратить баланс классов

In [28]:
X_train_kernel, _, y_train_kernel, _ = train_test_split(X_train, y_train, test_size=0.7)

In [32]:
%%time
model = RFFPipeline()
w = model.fit(X_train, y_train)
predictions = model.predict(X_test)
accuracy_score(y_test, predictions)

Sigma2 calculated 118.10371481760049
Weights calculated
(60000, 1000)
Phi matrix calculated


/Users/fess/opt/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


CPU times: user 57.3 s, sys: 2.87 s, total: 1min
Wall time: 25.8 s


0.8564

__Задание 2. (3 балла)__

Сравните подход со случайными признаками с обучением SVM на исходных признаках. Попробуйте вариант с обычным (линейным) SVM и с ядровым SVM. Ядровой SVM может очень долго обучаться, поэтому можно делать любые разумные вещи для ускорения: брать подмножество объектов из обучающей выборки, например.

Сравните подход со случайными признаками с вариантом, в котором вы понижаете размерность с помощью PCA и обучаете градиентный бустинг. Используйте одну из реализаций CatBoost/LightGBM/XGBoost, не забудьте подобрать число деревьев и длину шага.

Сделайте выводы — насколько идея со случайными признаками работает? Сравните как с точки зрения качества, так и с точки зрения скорости обучения и применения.

In [7]:
%%time 

linear_SVM = SVC(kernel='linear')
linear_SVM.fit(X_train, y_train)
linear_predictions = linear_SVM.predict(X_test)
accuracy_score(y_test, linear_predictions)

Wall time: 5min 20s


0.8464

In [8]:
%%time 

kernel_SVM = SVC()
kernel_SVM.fit(X_train_kernel, y_train_kernel)
kernel_predictions = kernel_SVM.predict(X_test)
accuracy_score(y_test, kernel_predictions)

Wall time: 1min 4s


0.8668

In [9]:
#pca = PCA(n_components=50)
#X_train_boost = pca.fit_transform(X_train)
#X_test_boost = pca.transform(X_test)

In [24]:
def objective(trial: optuna.Trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 20, 500),
        'eta': trial.suggest_float('eta', 0, 1),
        'silent': True,
        'random_seed': MAIN_SEED,
        'loss_function': 'MultiClass',
        'task_type': 'GPU'
    }

    model = CatBoostClassifier(**params)
    
    model.fit(X_train, y_train)
    result = model.predict(X_test)
    score = accuracy_score(y_test, result)
    return score

In [25]:
sampler = TPESampler(seed=MAIN_SEED)
study = optuna.create_study(direction='minimize', sampler=sampler)

In [30]:
study.optimize(objective, n_trials=100, show_progress_bar=True)
best_boost = study.best_params
best_boost

C:\ProgramData\Anaconda3\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()
100%|██████████| 100/100 [31:40<00:00, 19.00s/it]


{'n_estimators': 183, 'eta': 0.00011823253634721254}

In [31]:
params_boost = {
    'random_seed': MAIN_SEED,
    'loss_function': 'MultiClass',
    'task_type': 'GPU'
}

for key in best_boost:
    params_boost[key] = best_boost[key]

In [33]:
best_boost

{'n_estimators': 183, 'eta': 0.00011823253634721254}

In [32]:
%%time
boost_model = CatBoostClassifier(**params_boost)
boost_model.fit(X_train, y_train)
boost_predictions = boost_model.predict(X_test)
accuracy_score(y_test, boost_predictions)

0:	learn: 2.3021000	total: 45ms	remaining: 8.18s
1:	learn: 2.3015969	total: 86.9ms	remaining: 7.87s
2:	learn: 2.3010922	total: 129ms	remaining: 7.71s
3:	learn: 2.3005896	total: 170ms	remaining: 7.61s
4:	learn: 2.3001005	total: 212ms	remaining: 7.56s
5:	learn: 2.2995747	total: 253ms	remaining: 7.46s
6:	learn: 2.2990628	total: 297ms	remaining: 7.46s
7:	learn: 2.2985612	total: 1.4s	remaining: 30.6s
8:	learn: 2.2980443	total: 1.44s	remaining: 27.9s
9:	learn: 2.2975474	total: 1.49s	remaining: 25.7s
10:	learn: 2.2970698	total: 1.53s	remaining: 23.9s
11:	learn: 2.2965740	total: 1.57s	remaining: 22.4s
12:	learn: 2.2960771	total: 1.61s	remaining: 21.1s
13:	learn: 2.2955813	total: 1.66s	remaining: 20s
14:	learn: 2.2950896	total: 1.7s	remaining: 19s
15:	learn: 2.2945995	total: 1.74s	remaining: 18.2s
16:	learn: 2.2941130	total: 1.79s	remaining: 17.4s
17:	learn: 2.2936193	total: 1.83s	remaining: 16.8s
18:	learn: 2.2931253	total: 1.87s	remaining: 16.2s
19:	learn: 2.2926292	total: 1.91s	remaining: 15

0.7027

__Задание 3. (2 балла)__

Проведите эксперименты:
1. Помогает ли предварительное понижение размерности с помощью PCA? 
2. Как зависит итоговое качество от n_features? Выходит ли оно на плато при росте n_features?
3. Важно ли, какую модель обучать — логистическую регрессию или SVM?

In [ ]:
result = {}
for model in ['logreg', 'svm']:
    result_acc = []
    for n_features in [100, 300, 500, 700, 1000, 1500, 2000, 5000]:
        model = RFFPipeline()
        result_acc.appe


### Бонус

__Задание 4. (Максимум 2 балла)__

Как вы, должно быть, помните с курса МО-1, многие алгоритмы машинного обучения работают лучше, если признаки данных некоррелированы. Оказывается, что для RFF существует модификация, позволяющая получать ортогональные случайные признаки (Orthogonal Random Features, ORF). Об этом методе можно прочитать в [статье](https://proceedings.neurips.cc/paper/2016/file/53adaf494dc89ef7196d73636eb2451b-Paper.pdf). Реализуйте класс для вычисления ORF по аналогии с основным заданием. Обратите внимание, что ваш класс должен уметь работать со случаем n_features > new_dim (в статье есть замечание на этот счет). Проведите эксперименты, сравнивающие RFF и ORF, сделайте выводы.

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪

__Задание 5. (Максимум 2 балла)__

Поэкспериментируйте с функциями для вычисления новых случайных признаков. Не обязательно использовать косинус от скалярного произведения — можно брать знак от него, хэш и т.д. Придумайте побольше вариантов для генерации признаков и проверьте, не получается ли с их помощью добиваться более высокого качества. Также можете попробовать другой классификатор поверх случайных признаков, сравните результаты.

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪